In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_450x32x32_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_450x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
#train, valid = to_tf(train).cache(f'{tmp}/ours_32'), to_tf(valid).cache(f'{tmp}/ours_32')
train, valid = to_tf(train), to_tf(valid)

In [3]:
m2 = M_2()
m2.compile(optimizer='Nadam', loss='mae')
m2.build(input_shape=(None, 450, 32, 32, 3))
print(f'Flops per frame:{get_flops(m2, input_sig=[tf.TensorSpec([1, 450, 32, 32, 3])])/450:.0f}')
m2.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:502082
Model: "m_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           196672    
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_1 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  10272     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 32)           128       
_________________________________________________________________
sequential_3 (Sequential)    (None, 450)               33        
Total params: 248

In [ ]:
m2.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m2_32.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m2.load_weights('../weights/m2_32.h5')

In [4]:
m2.load_weights('../weights/m2_32.h5')

In [5]:
eval_on_dataset(test_set_CCNU, m2, 450, (32, 32), step=1, batch=32, save='../results/ours_32_conv_only_CCNU_CCNU.h5')
get_metrics('../results/ours_32_conv_only_CCNU_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [01:34<00:00,  1.89it/s]


{'Sliding window': {'MAE': 1.531, 'RMSE': 5.26, 'R': 0.87514},
 'Whole video': {'MAE': 1.019, 'RMSE': 4.257, 'R': 0.91149}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, m2, 450, (32, 32), step=1, batch=32, save='../results/ours_32_conv_only_CCNU_CCNU_rPPG.h5')
get_metrics('../results/ours_32_conv_only_CCNU_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [00:20<00:00,  2.67it/s]


{'Sliding window': {'MAE': 1.74, 'RMSE': 6.329, 'R': 0.81023},
 'Whole video': {'MAE': 0.801, 'RMSE': 2.292, 'R': 0.97249}}

In [7]:
eval_on_dataset(test_set_PURE, m2, 450, (32, 32), step=1, batch=1, save='../results/ours_32_conv_only_CCNU_PURE.h5')
get_metrics('../results/ours_32_conv_only_CCNU_PURE.h5', use_filter=False)

100%|██████████| 59/59 [00:09<00:00,  6.55it/s]


{'Sliding window': {'MAE': 0.636, 'RMSE': 2.768, 'R': 0.99274},
 'Whole video': {'MAE': 0.376, 'RMSE': 0.497, 'R': 0.99979}}

In [8]:
eval_on_dataset(test_set_UBFC_rPPG2, m2, 450, (32, 32), step=1, batch=1, save='../results/ours_32_conv_only_CCNU_UBFC.h5')
get_metrics('../results/ours_32_conv_only_CCNU_UBFC.h5', use_filter=False)

100%|██████████| 42/42 [00:06<00:00,  6.72it/s]


{'Sliding window': {'MAE': 1.454, 'RMSE': 3.776, 'R': 0.97602},
 'Whole video': {'MAE': 0.838, 'RMSE': 1.289, 'R': 0.99759}}